# EWS 1 — Revolving Credit (Rolling Features via rolling.py)

This notebook builds an **Early Warning System (EWS)** for a **revolving credit portfolio** (e.g., credit cards) using **rolling behavioural features** generated via `src/features/rolling.py`.

**What you get**
- Synthetic account-month panel data (safe for GitHub)
- Rolling features (recency, worst-case, persistence, volatility)
- Rule-based alerting and monitoring views


In [ ]:
import sys
from pathlib import Path

# Make repo src importable
REPO_ROOT = Path('.').resolve()
SRC = REPO_ROOT / 'src'
if str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

np.random.seed(42)
pd.set_option('display.max_columns', 200)


In [ ]:
from features.rolling import RollingSpec, add_rolling_features

## 1) Simulate revolving credit panel (synthetic)

In [ ]:
def simulate_revolving_panel(n_accounts=6000, n_months=24, start="2023-01-31") -> pd.DataFrame:
    start_date = pd.to_datetime(start)
    dates = pd.date_range(start_date, periods=n_months, freq="M")

    account_id = np.arange(n_accounts)
    product = np.random.choice(["Classic", "Rewards", "Premium"], size=n_accounts, p=[0.55, 0.35, 0.10])
    channel = np.random.choice(["Branch", "Online", "Partner"], size=n_accounts, p=[0.30, 0.55, 0.15])
    risk_band = np.random.choice(["A", "B", "C", "D"], size=n_accounts, p=[0.35, 0.35, 0.20, 0.10])

    band_limit_mu = {"A": 7000, "B": 5000, "C": 3500, "D": 2500}
    credit_limit = np.array([max(800, np.random.normal(band_limit_mu[b], 900)) for b in risk_band])
    credit_limit = np.round(credit_limit, 0)

    band_stress = {"A": 0.10, "B": 0.20, "C": 0.32, "D": 0.45}
    base_stress = np.array([np.random.beta(2, 10) + band_stress[b] for b in risk_band])
    base_stress = np.clip(base_stress, 0, 1)

    util_prev = np.random.beta(2, 5, size=n_accounts)
    pay_prev = np.clip(np.random.normal(0.90, 0.12, size=n_accounts), 0, 1)
    dpd_prev = np.zeros(n_accounts, dtype=int)

    rows = []
    for t, d in enumerate(dates):
        macro = 0.05 * np.sin(2 * np.pi * (t / 12))

        util = 0.65 * util_prev + 0.25 * base_stress + 0.10 * np.random.beta(2, 5, size=n_accounts) + macro
        util = np.clip(util, 0, 1.4)

        pay = (
            0.60 * pay_prev
            + 0.35 * (1 - base_stress)
            - 0.20 * np.maximum(util - 0.8, 0)
            + 0.05 * np.random.normal(0, 1, size=n_accounts)
        )
        pay = np.clip(pay, 0, 1.2)

        p_miss = 0.03 + 0.25 * (1 - np.clip(pay, 0, 1)) + 0.20 * base_stress + 0.10 * np.maximum(util - 0.9, 0)
        p_miss = np.clip(p_miss, 0, 0.9)
        missed_payment = (np.random.rand(n_accounts) < p_miss).astype(int)

        dpd = dpd_prev.copy()
        migrate_up = missed_payment == 1
        dpd[migrate_up & (dpd_prev == 0)] = 30
        dpd[migrate_up & (dpd_prev == 30)] = 60
        dpd[migrate_up & (dpd_prev == 60)] = 90
        dpd[migrate_up & (dpd_prev == 90)] = 90

        cure = (missed_payment == 0) & (pay > 0.6)
        dpd[cure & (dpd_prev == 30)] = 0
        dpd[cure & (dpd_prev == 60)] = 30
        dpd[cure & (dpd_prev == 90)] = 60

        balance = util * credit_limit + np.random.normal(0, 120, size=n_accounts)
        balance = np.clip(balance, 0, None)

        score = (
            -3.8
            + 1.8 * (dpd >= 30).astype(int)
            + 2.4 * (dpd >= 60).astype(int)
            + 3.0 * (dpd >= 90).astype(int)
            + 2.0 * np.maximum(util - 0.85, 0)
            + 2.2 * np.maximum(0.7 - np.clip(pay, 0, 1), 0)
            + 1.2 * missed_payment
            + 1.3 * base_stress
            + 0.6 * macro
        )
        p_default = 1 / (1 + np.exp(-score))
        default_next_3m = (np.random.rand(n_accounts) < p_default).astype(int)

        rows.append(pd.DataFrame({
            "account_id": account_id,
            "date": d,
            "product": product,
            "channel": channel,
            "risk_band": risk_band,
            "credit_limit": credit_limit,
            "balance": balance,
            "utilization": balance / credit_limit,
            "payment_ratio": pay,
            "missed_payment": missed_payment,
            "dpd_bucket": dpd,
            "default_next_3m": default_next_3m,
        }))

        util_prev = np.clip(util, 0, 1.4)
        pay_prev = np.clip(pay, 0, 1.2)
        dpd_prev = dpd

    return pd.concat(rows, ignore_index=True).sort_values(["account_id","date"]).reset_index(drop=True)

df = simulate_revolving_panel()
df.head()

## 2) Rolling features via `rolling.py`

In [ ]:
specs = [
    RollingSpec("utilization", 3, "mean", "util_3m_avg"),
    RollingSpec("utilization", 6, "max",  "util_6m_max"),
    RollingSpec("utilization", 6, "mean", "util_6m_avg"),
    RollingSpec("payment_ratio", 3, "mean", "pay_3m_avg"),
    RollingSpec("payment_ratio", 6, "min",  "pay_6m_min"),
    RollingSpec("payment_ratio", 6, "std",  "pay_6m_std"),
    RollingSpec("payment_ratio", 6, "mean", "pay_6m_avg"),
    RollingSpec("missed_payment", 6, "sum", "miss_6m_sum"),
    RollingSpec("dpd_bucket", 6, "any_ge", "any_dpd30_6m", threshold=30),
]

df_feat = add_rolling_features(df, group_cols="account_id", date_col="date", specs=specs)
df_feat["util_trend_3m_vs_6m"] = df_feat["util_3m_avg"] - df_feat["util_6m_avg"]
feat_cols = [s.out_col for s in specs] + ["util_trend_3m_vs_6m"]
model_df = df_feat.dropna(subset=feat_cols + ["default_next_3m"]).copy()
model_df.shape

## 3) Rule-based alerts + monitoring view

In [ ]:
TH_UTIL_MAX = 0.95
TH_PAY_AVG  = 0.65
TH_MISS_6M  = 2
TH_UTIL_TREND = 0.12

model_df["alert_rule"] = (
    ((model_df["util_6m_max"] >= TH_UTIL_MAX) & (model_df["pay_3m_avg"] <= TH_PAY_AVG))
    | (model_df["miss_6m_sum"] >= TH_MISS_6M)
    | (model_df["any_dpd30_6m"] == 1)
    | (model_df["util_trend_3m_vs_6m"] >= TH_UTIL_TREND)
).astype(int)

monthly = (
    model_df.groupby("date")
    .agg(accounts=("account_id","nunique"),
         alert_rate=("alert_rule","mean"),
         default_rate=("default_next_3m","mean"))
)

plt.figure(figsize=(10,4))
plt.plot(monthly.index, monthly["alert_rate"], label="Alert rate")
plt.plot(monthly.index, monthly["default_rate"], label="Default (next 3m) rate")
plt.title("Revolving EWS: Alert Rate vs Default Rate")
plt.xlabel("Month")
plt.ylabel("Rate")
plt.legend()
plt.tight_layout()
plt.show()

monthly.tail()